# unet implimentation in renetal vessel dataset

In [ ]:
!pip install numpy==1.16.4

In [ ]:
import numpy as np

seed=42

np.random.seed = seed

In [ ]:
from sklearn.model_selection import train_test_split
from glob import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.transform import resize
from keras import regularizers

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.io import imread
import numpy as np
from keras.callbacks import History
import matplotlib.pyplot as plt


print('okay')

In [ ]:
# Set some parameters
height = 64*3
width = 64*3
IMG_CHANNELS = 1

seed = 42

In [ ]:
mask=np.load(r'../input/brats-2013/npy_file_t1c/npy_file_t1c/dataset/dataset.npy',allow_pickle=True)
dataset=np.load(r'../input/brats-2013/npy_file_t1c/npy_file_t1c/masks/mask.npy',allow_pickle=True)

In [ ]:
mask[1].shape

In [ ]:
image_set=[]

mask_set=[]

for i in range(1000):
    image_set.append(cv2.resize(dataset[i],(width,height)))
    
    mask_set.append(cv2.resize(mask[i],(width,height)))

In [ ]:
mask_set=np.array(mask_set)

image_set=np.array(image_set)

In [ ]:
batchsize=8

print('okay')

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

print('okay')

In [ ]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

In [ ]:
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
# Build U-Net model
inputs = Input((height, width, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(rate=0.1) (c1)
c1 = Conv2D(8, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(rate=0.1) (c2)
c2 = Conv2D(16, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(rate=0.2) (c3)
c3 = Conv2D(32, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(rate=0.2) (c4)
c4 = Conv2D(64, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(rate=0.3) (c5)
c5 = Conv2D(128, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(rate=0.2) (c6)
c6 = Conv2D(64, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001),
            kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(rate=0.2) (c7)
c7 = Conv2D(32, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(rate=0.1) (c8)
c8 = Conv2D(16, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(rate=0.1) (c9)
c9 = Conv2D(8, (3, 3), activation='elu', kernel_regularizer=regularizers.l2(0.00001),
            kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = (Model(inputs=[inputs], outputs=[outputs]))

#model.summary()

print('okay')

In [ ]:
#pngs_mask[0]

In [ ]:
image_set[0].shape

In [ ]:
mask_set.shape

In [ ]:
#image_set[0]

In [ ]:
image_set = image_set.reshape(-1,height,width,1)
mask_set = mask_set.reshape(-1,height,width,1)

In [ ]:
#image_set=np.expand_dims(image_set,axis=3)
#mask_set=np.expand_dims(mask_set,axis=3)

In [ ]:
# Split train and valid
#X_train, X_valid, X_feat_train, X_feat_valid, y_train, y_valid = train_test_split(X, X_feat, y, test_size=0.15, random_state=42)

In [ ]:
# Normalize X_feat
#x_feat_mean = X_feat_train.mean(axis=0, keepdims=True)
#x_feat_std = X_feat_train.std(axis=0, keepdims=True)
#X_feat_train -= x_feat_mean
#X_feat_train /= x_feat_std

#X_feat_valid -= x_feat_mean
#X_feat_valid /= x_feat_std

In [ ]:
image_set.shape

In [ ]:
#callbacks = [
    #EarlyStopping(patience=5, verbose=1),
    #ReduceLROnPlateau(patience=3, verbose=1),
    #ModelCheckpoint('model-tgs-salt-1.h5', verbose=1, save_best_only=True, save_weights_only=True)]

#results = model.fit({'img': X_train, 'feat': X_feat_train}, y_train, batch_size=16, epochs=50, callbacks=callbacks,
                    #validation_data=({'img': X_valid, 'feat': X_feat_valid}, y_valid))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
output=model.fit(x=image_set, y=mask_set, batch_size=None, epochs=100, callbacks=None, validation_split=0.3, 
          validation_data=None,steps_per_epoch=700//batchsize,validation_steps=300)

In [ ]:
#model.evaluate({'img': X_valid, 'feat': X_feat_valid}, y_valid, verbose=1)


In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
'''def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)
thres = np.linspace(0.25, 0.75, 20)
thres_ioc = [iou_metric_batch(y_valid, np.int32(preds_val > t)) for t in tqdm_notebook(thres)]'''

In [ ]:
print('training complete')

In [ ]:
model.save_weights('u_net_weights.h5')

In [ ]:
import cv2
import random

num=random.randint(0,100)

#name=r'../input/mri/mri/image/renamedimage/id_' + str(num) + '.png'

np.seterr(divide='ignore', invalid='ignore')

#test_img = cv2.imread(name,0)

test_img=image_set[num]

test_img = test_img/np.max(test_img)

#test_image= image.img_to_array(test_image)

test_img = resize(test_img,(height,width,1))

test_img.shape

In [ ]:
test_img = np.expand_dims(test_img, axis=0)
test_img.shape

In [ ]:
results = model.predict(test_img,batch_size=None, verbose=0, steps=1)

results.shape

In [ ]:
from matplotlib import pyplot as plt

results=np.reshape(results,(height,width))

results.shape

In [ ]:
test_image=np.reshape(test_img,(height,width))

In [ ]:
#name_actual_mask=r'../input/mri/mri/mask/markedrenamed/id_' + str(num) + '.png'

#actual_mask=cv2.imread(name_actual_mask)

actual_mask=mask_set[num]

In [ ]:
# Predict on train, val and test
'''preds_train = model.predict({'img': X_train, 'feat': X_feat_train}, verbose=1)
preds_val = model.predict({'img': X_valid, 'feat': X_feat_valid}, verbose=1)
preds_test = model.predict({'img': X_test, 'feat': X_feat_test}, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)'''

In [ ]:
actual_mask.shape

In [ ]:
op=results>0.5

In [ ]:
print(num)

plt.imshow(np.reshape(test_img,(height,width)),cmap='seismic')
plt.title('actual image')
plt.show()

plt.imshow(op,cmap='seismic')
plt.title('predicted mask')
plt.show()

plt.imshow(np.reshape(actual_mask,(height,width)),cmap='seismic')
plt.title('actual mask')
plt.show()